In [1]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
!apt-get update
!apt-get install unzip

Hit:1 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
0% [Waiting for headers]

Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.1).
0 upgraded, 0 newly installed, 0 to remove and 69 not upgraded.


In [4]:
import os
from getpass import getpass
from kaggle.api.kaggle_api_extended import KaggleApi

import pandas as pd
import shutil
from tqdm.auto import tqdm
import ast
import pybboxes as pbx

import ultralytics
from ultralytics import YOLO

import random

In [6]:
os.environ['KAGGLE_USERNAME'] = input()
os.environ['KAGGLE_KEY'] = getpass()

In [7]:
api = KaggleApi()
api.authenticate()

In [9]:
!rm -rf ./datasets
!kaggle datasets download -d vbookshelf/v2-balloon-detection-dataset
!mkdir ./datasets
!unzip /workspace/object-detection-balloons/v2-balloon-detection-dataset.zip -d ./datasets/balloons
!rm -rf /workspace/object-detection-balloons/v2-balloon-detection-dataset.zip

100%|██████████████████████████████████████| 47.5M/47.5M [00:01<00:00, 36.3MB/s]
100%|██████████████████████████████████████| 47.5M/47.5M [00:01<00:00, 27.9MB/s]
Archive:  /workspace/object-detection-balloons/v2-balloon-detection-dataset.zip
  inflating: ./datasets/balloons/Arial.ttf  
  inflating: ./datasets/balloons/balloon-data.csv  
  inflating: ./datasets/balloons/images/10464445726_6f1e3bbe6a_k.jpg  
  inflating: ./datasets/balloons/images/12037308314_e16fb3a0f7_k.jpg  
  inflating: ./datasets/balloons/images/120853323_d4788431b9_b.jpg  
  inflating: ./datasets/balloons/images/12288043903_fe1ea17a4e_k.jpg  
  inflating: ./datasets/balloons/images/12288355124_5e340d3de3_k.jpg  
  inflating: ./datasets/balloons/images/12288446656_2c6a90e6f5_k.jpg  
  inflating: ./datasets/balloons/images/126700562_8e27720147_b.jpg  
  inflating: ./datasets/balloons/images/1297451346_5b92bdac08_b.jpg  
  inflating: ./datasets/balloons/images/14321263043_b76ef054d3_k.jpg  
  inflating: ./datasets/bal

In [10]:
csv_url = "/workspace/object-detection-balloons/datasets/balloons/balloon-data.csv"
balloon_df = pd.read_csv(csv_url)
print(balloon_df.shape)
balloon_df.head()

(74, 5)


,fname,height,width,bbox,num_balloons
0,34020010494_e5cb88e1c4_k.jpg,1536,2048,"[{'xmin': 994, 'ymin': 619, 'xmax': 1445, 'yma...",1
1,25899693952_7c8b8b9edc_k.jpg,1365,2048,"[{'xmin': 135, 'ymin': 115, 'xmax': 811, 'ymax...",1
2,24362039530_b151b41a52_k.jpg,2048,1536,"[{'xmin': 579, 'ymin': 487, 'xmax': 920, 'ymax...",1
3,18849792632_aad23ad513_k.jpg,2048,1536,"[{'xmin': 534, 'ymin': 365, 'xmax': 871, 'ymax...",10
4,17178818589_16e58fc1e5_k.jpg,1536,2048,"[{'xmin': 748, 'ymin': 195, 'xmax': 1425, 'yma...",7


## Train-Test split

In [11]:
# randomly assing 20% of the examples to the validation set

random.seed(2077)

balloon_df["is_val"] = [random.uniform(0, 1) <= 0.2 for _ in range(len(balloon_df))]

In [12]:
# check size of the validation set

balloon_df.loc[balloon_df["is_val"] == True].shape

(16, 6)

In [28]:
len(os.listdir("/workspace/object-detection-balloons/datasets/balloons/labels/train"))

58

In [13]:
# define directory structure to accomodate train-validation split

os.rename("/workspace/object-detection-balloons/datasets/balloons/images", "/workspace/object-detection-balloons/datasets/balloons/train")
os.makedirs("/workspace/object-detection-balloons/datasets/balloons/images", exist_ok=True)
shutil.move("/workspace/object-detection-balloons/datasets/balloons/train", "/workspace/object-detection-balloons/datasets/balloons/images/train")

os.makedirs("/workspace/object-detection-balloons/datasets/balloons/labels/train", exist_ok=True)

img_val_dir = "/workspace/object-detection-balloons/datasets/balloons/images/val"
os.mkdir(img_val_dir)

lbs_val_dir = "/workspace/object-detection-balloons/datasets/balloons/labels/val"
os.mkdir(lbs_val_dir)

In [14]:
# from images from train to validation folder

for i, row in balloon_df.loc[balloon_df["is_val"] == True].iterrows():
    src_path = os.path.join("/workspace/object-detection-balloons/datasets/balloons/images/train", row["fname"])
    dst_path = os.path.join("/workspace/object-detection-balloons/datasets/balloons/images/val", row["fname"])
    shutil.move(src_path, dst_path)

In [15]:
# function to create annotations as .txt files according to the YOLO format

def create_yolo_labels(row):
    dir = "/workspace/object-detection-balloons/datasets/balloons/labels/train/"
    bboxes = ast.literal_eval(row["bbox"])
    for bbox in bboxes:
        bbox = bbox.values()
        w, h = row["width"], row["height"]
        bbox_yolo = pbx.convert_bbox(bbox, from_type="voc", to_type="yolo", image_size=(w, h))
        bbox_yolo = " ".join([str(bb) for bb in bbox_yolo])
        file_path = dir + row["fname"].replace(".jpg", ".txt")
        with open(file_path, "a") as f:
            f.write(f"0 {bbox_yolo}\n")

# create_yolo_labels(balloon_df.iloc[0])

In [16]:
# create annotations

for i, row in tqdm(balloon_df.iterrows()):
    create_yolo_labels(row)

0it [00:00, ?it/s]

In [17]:
# move annotations from train to validation folder

for i, row in balloon_df.loc[balloon_df["is_val"] == True].iterrows():
    src_path = os.path.join("/workspace/object-detection-balloons/datasets/balloons/labels/train", row["fname"].replace(".jpg", ".txt"))
    dst_path = os.path.join("/workspace/object-detection-balloons/datasets/balloons/labels/val", row["fname"].replace(".jpg", ".txt"))
    shutil.move(src_path, dst_path)

In [18]:
ultralytics.checks()

Ultralytics YOLOv8.0.187 🚀 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (Tesla V100-FHHL-16GB, 16151MiB)
Setup complete ✅ (32 CPUs, 94.3 GB RAM, 1.3/20.0 GB disk)


## Baseline

In [27]:
img_path_base = "/workspace/object-detection-balloons/datasets/balloons/images/val"
val_img_paths = [os.path.join(img_path_base, fname) for fname in os.listdir(img_path_base)]
print(len(val_img_paths))
val_img_paths[:3]

16


['/workspace/object-detection-balloons/datasets/balloons/images/val/7178882742_f090f3ce56_k.jpg',
 '/workspace/object-detection-balloons/datasets/balloons/images/val/6483318883_21facf57cd_b.jpg',
 '/workspace/object-detection-balloons/datasets/balloons/images/val/5560377994_cb597a4af5_b.jpg']

In [34]:
yolo_base = YOLO('yolov8n.pt')
preds = yolo_base.predict(val_img_paths, save=True, project="preds", name="baseline") 


0: 640x640 9 persons, 1 bowl, 1 orange, 3 dining tables, 1: 640x640 6 persons, 1 bowl, 2: 640x640 6 persons, 1 bicycle, 3 kites, 3: 640x640 1 sports ball, 4: 640x640 2 apples, 5: 640x640 1 person, 1 frisbee, 1 bowl, 2 apples, 6: 640x640 2 kites, 7: 640x640 10 persons, 1 sports ball, 1 chair, 8: 640x640 2 kites, 9: 640x640 2 vases, 10: 640x640 1 person, 2 cars, 5 sports balls, 11: 640x640 2 sports balls, 12: 640x640 1 person, 1 traffic light, 13: 640x640 1 sports ball, 14: 640x640 5 persons, 1 motorcycle, 1 backpack, 15: 640x640 1 person, 2 sports balls, 1 kite, 25.3ms
Speed: 4.0ms preprocess, 1.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to preds/baseline


In [35]:
metrics = yolo_base.val(data="/workspace/object-detection-balloons/balloons.yaml", split="val")

Ultralytics YOLOv8.0.187 🚀 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (Tesla V100-FHHL-16GB, 16151MiB)
val: Scanning /workspace/object-detection-balloons/datasets/balloons/labels/val.cache... 16 images, 0 backgrounds, 0 corrupt: 100%|██████████| 16/16 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.99s/it]
                   all         16         68     0.0244      0.162     0.0143    0.00879
                person         16         68     0.0244      0.162     0.0143    0.00879
Speed: 0.2ms preprocess, 1.8ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/val


## Finetuning

In [39]:
yolo_finetuned = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)
dataset_yaml_path = "/workspace/object-detection-balloons/balloons.yaml"

# Use the model
results = yolo_finetuned.train(data=dataset_yaml_path, epochs=50)  # train the model

Ultralytics YOLOv8.0.187 🚀 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (Tesla V100-FHHL-16GB, 16151MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/workspace/object-detection-balloons/balloons.yaml, epochs=50, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, for

In [42]:
!rm -rf /workspace/object-detection-balloons/preds/finetuned
preds = yolo_finetuned.predict(val_img_paths, save=True, project="preds", name="finetuned") 


0: 640x640 2 balloons, 1: 640x640 3 balloons, 2: 640x640 13 balloons, 3: 640x640 3 balloons, 4: 640x640 4 balloons, 5: 640x640 7 balloons, 6: 640x640 1 balloon, 7: 640x640 1 balloon, 8: 640x640 1 balloon, 9: 640x640 6 balloons, 10: 640x640 8 balloons, 11: 640x640 2 balloons, 12: 640x640 6 balloons, 13: 640x640 3 balloons, 14: 640x640 2 balloons, 15: 640x640 8 balloons, 23.2ms
Speed: 2.8ms preprocess, 1.4ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)
Results saved to preds/finetuned


In [43]:
metrics = yolo_finetuned.val(data="/workspace/object-detection-balloons/balloons.yaml", split="val")

Ultralytics YOLOv8.0.187 🚀 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (Tesla V100-FHHL-16GB, 16151MiB)
val: Scanning /workspace/object-detection-balloons/datasets/balloons/labels/val.cache... 16 images, 0 backgrounds, 0 corrupt: 100%|██████████| 16/16 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.12it/s]
                   all         16         68      0.789      0.809      0.844      0.716
Speed: 0.2ms preprocess, 1.8ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to runs/detect/val4


: 